In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn import preprocessing
import gc

In [2]:
import pandas as pd
data = pd.read_csv('../data/housing_price.csv')
pre_data = pd.read_csv('../data/pre_data.csv')
print(data.head(2))


   Id  BATHRM  HF_BATHRM           HEAT AC  ROOMS  BEDRM     AYB  YR_RMDL  \
0   1       3          2  Hot Water Rad  Y     10      5  1913.0      NaN   
1   2       3          1      Warm Cool  Y      8      3  1908.0   2006.0   

    EYB  ...        NATIONALGRID   LATITUDE  LONGITUDE  ASSESSMENT_NBHD  \
0  1972  ...  18S UJ 23157 09248  38.914331 -77.039715       Old City 2   
1  1967  ...  18S UJ 23147 09324  38.915018 -77.039844       Old City 2   

  ASSESSMENT_SUBNBHD CENSUS_TRACT CENSUS_BLOCK    WARD QUADRANT fold  
0   040 D Old City 2         4201  004201 2006  Ward 2       NW    1  
1   040 D Old City 2         4201  004201 2005  Ward 2       NW    3  

[2 rows x 37 columns]


In [3]:
# Encode
def encod(col):
    le = preprocessing.LabelEncoder()
    if (col == 'AC'):
        le.fit(['Y','N'])
        data[col] = le.transform(data[col])
        pre_data[col] = le.transform(pre_data[col])
    elif(col == 'GRADE'):
        le.fit(['Low Quality', 'Fair Quality', 'Average', 'Above Average', 'Good Quality', 'Very Good', 'Excellent', 'Superior', 'Exceptional-A', 'Exceptional-B', 'Exceptional-C', 'Exceptional-D'])
        data[col] = le.transform(data[col])
        pre_data[col] = le.transform(pre_data[col])
    elif(col == 'CNDTN'):
        le.fit(['Poor', 'Fair', 'Average', 'Good', 'Very Good','Excellent'])
        data[col] = le.transform(data[col])
        pre_data[col] = le.transform(pre_data[col])
    else:
        le = preprocessing.LabelEncoder()
        le.fit(data[col])
        data[col] = le.transform(data[col]) 
        le.fit(pre_data[col])
        pre_data[col] = le.transform(pre_data[col]) 
def toordinal(x):
    return x.toordinal()

In [4]:
import numpy as np
#index = data['KITCHENS'].index[data['KITCHENS'].apply(np.isnan)]
index = 37475

data.loc [index, 'KITCHENS']=1
data.loc [data['AC'].index[data['AC'].apply(lambda x: x=='0')], 'AC']='Y'

In [5]:


data['SALEDATE'] = data['SALEDATE'].astype('datetime64[ns]').apply(toordinal) 
pre_data['SALEDATE'] = pre_data['SALEDATE'].astype('datetime64[ns]').apply(toordinal) 

# deleted FULLADDRESS, NATIONALGRID
temp_drop=['FULLADDRESS','NATIONALGRID','ASSESSMENT_SUBNBHD','CENSUS_BLOCK','QUADRANT','LATITUDE','LONGITUDE']
for col in temp_drop:
    data = data.drop(col,1)
    pre_data = pre_data.drop(col,1)
    
cols = ['HEAT','AC','STYLE','STRUCT','GRADE','CNDTN','EXTWALL','ROOF','INTWALL','ASSESSMENT_NBHD','WARD']


for col in cols:
    print(col)
    encod(col)

HEAT
AC
STYLE
STRUCT
GRADE
CNDTN
EXTWALL
ROOF
INTWALL
ASSESSMENT_NBHD
WARD


In [6]:
drop_org=['STORIES','YR_RMDL','AYB']
data[data['KITCHENS'].isnull()]['KITCHENS'] = 1
for col in drop_org:
    data = data.drop(col,1)


In [7]:
import xgboost as xgb
def rmsle(y, y0):
    return np.sqrt(np.mean(np.square(np.log1p(y) - np.log1p(y0))))

def train_test_with_fold(fold, max_dep):
    clf1 = RandomForestClassifier(n_estimators=500,max_depth=None)

    gc.collect()
    train1 = data[data['fold']!=fold]
    
    test1 = data[data['fold']==fold]

    
    train1 = train1.drop('fold',1)
   
    test1 = test1.drop('fold',1)


    gc.collect()
    clf_rf1 = clf1.fit( train1.drop('PRICE',1),train1['PRICE'])
    y_pred1 = clf_rf1.predict(test1.drop('PRICE',1))
    score1 = rmsle(y_pred1,test1['PRICE'])
    print('original data: '+str(score1))
    del clf_rf1, train1, test1
    gc.collect()
    
    train2 = pre_data[pre_data['fold']!=fold]
    test2 = pre_data[pre_data['fold']==fold]
    train2 = train2.drop('fold',1)
    test2 = test2.drop('fold',1)
    gc.collect()
    clf_rf2 = clf1.fit( train2.drop('PRICE',1),train2['PRICE'])
    y_pred2 = clf_rf2.predict(test2.drop('PRICE',1))
    

    score2 = rmsle(y_pred2, test2['PRICE'])
    print('pre data: '+str(score2))
    
def xgb_train_test_with_fold(fold, max_dep):
    n = 10
    clf_xgb = xgb.XGBClassifier(
        max_depth=max_dep,
        n_estimators=n,
        subsample=0.5,
        learning_rate=0.1
        )
    clf_xgb2 = xgb.XGBClassifier(
        max_depth=max_dep,
        n_estimators=n,
        subsample=0.5,
        learning_rate=0.1
        )
    
    train1 = data[data['fold']!=fold]
    train2 = pre_data[pre_data['fold']!=fold]
    
    test1 = data[data['fold']==fold]
    test2 = pre_data[pre_data['fold']==fold]
    
    train1 = train1.drop('fold',1)
    train2 = train2.drop('fold',1)
    test1 = test1.drop('fold',1)
    test2 = test2.drop('fold',1)

    clf_rf1 = clf_xgb.fit( train1.drop('PRICE',1),train1['PRICE'])
    y_pred1 = clf_rf1.predict(test1.drop('PRICE',1))
    score1 = rmsle(y_pred1,test1['PRICE'])
    print('original data: '+str(score1))
    
    clf_rf2 = clf_xgb2.fit( train2.drop('PRICE',1),train2['PRICE'])

    y_pred2 = clf_rf2.predict(test2.drop('PRICE',1))
    
    score2 = rmsle(y_pred2, test2['PRICE'])
    print('pre data: '+str(score2))

In [8]:

for i in range(1,6):
    print(i)
    gc.collect()
    train_test_with_fold(i,10)
    #xgb_train_test_with_fold(i,10)

1


MemoryError: could not allocate 2980577280 bytes

In [ ]:
for i in range(1,6):
    print(i)
    #xgb_train_test_with_fold(i,10)

In [ ]:
train = data[data['fold']==2]


In [ ]:
clf_xgb = xgb.XGBClassifier(
    max_depth=2,
    n_estimators=500,
    subsample=0.5,
    learning_rate=0.1
    )
clf_xgb.fit(X,y)